In [1]:
import keras
model = keras_model = keras.models.load_model("mnist_cnn_allconv.h5")

Using TensorFlow backend.


In [2]:
from keras import backend as K
from keras.utils.np_utils import convert_kernel
import tensorflow as tf
ops = []
for layer in model.layers:
    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
        original_w = K.get_value(layer.W)
        converted_w = convert_kernel(original_w)
        ops.append(tf.assign(layer.W, converted_w).op)

In [3]:
K.get_session().run(ops)

[None, None]

In [ ]:
model.save_weights('mnist_cnn_allconv_tensorflow.h5')